In [1]:
import json
import pandas as pd
import numpy as np
import ast
import spacy
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', None)

/home/an/anaconda3/envs/deep_learning_env_2/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-07-12 08:49:45.778208: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-07-12 08:49:46.832142: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-07-12 08:49:46.833811: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-12 08:49:46.834003: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock

In [ ]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict

In [ ]:
from utils.track_1_kp_matching import *

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
import torch

In [ ]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader

First with top 10 args per category labelling of test set

# Load Test dataset

In [ ]:
df = pd.read_pickle(f"../data/Evaluation/test_data.pkl")

In [ ]:
df

In [ ]:
df['label'].value_counts()

In [ ]:
# df = df[df['topic'] == 'Hotels']
# df = df[df['topic'] == 'Restaurants']
# df = df[df['topic'] == 'Arts & Entertainment']
# df = df[df['topic'] == 'Automotive']
# df = df[df['topic'] == 'Beauty & Spas']

In [ ]:
# df = df[df['isMultiAspect'] == True]

In [ ]:
def prepare_comment_kp_label_input(df):
    df['num_of_aspects'] = df['comment'].str.len()
    df = df.rename(columns={'comment_id': 'comment_id_sent'})
    df['comment_id'] = df['comment_id_sent'] + "_" + df['num_of_aspects'].astype(str)
    
    comment_df = df[['comment_id', 'topic', 'full_comment', 'num_of_aspects',
                'aspects_x', 'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes']]
    comment_df = comment_df.drop_duplicates(subset=['comment_id']).reset_index(drop=True)
    comment_df.columns = [col.replace("_x", "") for col in comment_df.columns]
    
    kp_df = df[['key_point_id', 'topic', 'full_key_point',
           'aspects_y', 'opinions_y', 'opinion_aspect_pairs_y', 'sentiments_y', 'attributes']]
    kp_df = kp_df.drop_duplicates(subset=['key_point_id']).reset_index(drop=True)
    kp_df = kp_df.rename(columns={'full_key_point': 'key_point'})
    kp_df.columns = [col.replace("_y", "") for col in kp_df.columns]
    kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].apply(lambda x: ast.literal_eval(x))
    kp_df['sentiments'] = kp_df['sentiments'].apply(lambda x: ast.literal_eval(x))
    
    labels_df = df[['comment_id', 'key_point_id', 'label']]
    
    return comment_df, kp_df, labels_df

# Comment-KP ABSA Matching

In [ ]:
def comment_kp_absa_match(comment_df, kp_df):    
    comment_df = comment_df.explode(['aspects', 'opinions', 'opinion_aspect_pairs', 'sentiments', 'attributes'])
    kp_df = kp_df.explode(['opinion_aspect_pairs', 'sentiments'])
    
    # Matching based on ABSA predictions of comment and KPs
    sent_kp_df = comment_df.astype(str).merge(kp_df.astype(str), how='left', on=['topic']).dropna(subset=['key_point'])
    sent_kp_df = sent_kp_df.reset_index(drop=True)
    # Label the matching/non-matching pairs
    sent_kp_df.loc[
        (sent_kp_df['aspects_x'] == sent_kp_df['aspects_y']) & (sent_kp_df['sentiments_x'] == sent_kp_df['sentiments_y']),
        'score'
    ] = 1
    sent_kp_df.loc[
        (sent_kp_df['sentiments_x'] != sent_kp_df['sentiments_y']),
        'score'
    ] = 0
    sent_kp_df = implicit_aspects_matching(sent_kp_df)
    
    predictions_df = sent_kp_df.groupby(['topic', 'comment_id', 'opinion_aspect_pairs_x']).apply(select_best_kp_per_opinion).reset_index(drop=True)
    predictions_df = predictions_df.drop_duplicates(subset=['comment_id', 'key_point_id'])
    
    return predictions_df

In [ ]:
def select_best_kp_per_opinion(grp):
    best_match = grp.sort_values(by=['score'], ascending=False)
    best_match = best_match.iloc[0]
    return best_match

Comment/KP matching on implicit aspects

In [ ]:
import spacy
spacy.require_gpu()

In [ ]:
def implicit_aspects_matching(sent_kp_df):
    sent_kp_df['predicted_by_cosine'] = False
    
    implicit_match_analyze_mask = pd.isnull(sent_kp_df['score'])
    sent_kp_df[implicit_match_analyze_mask] = sent_kp_df[implicit_match_analyze_mask].apply(calculate_aspects_semantic_similarity, axis=1)
    
    return sent_kp_df

In [ ]:
unfound_tokens = []
def calculate_aspects_semantic_similarity(row):
    import warnings
    warnings.filterwarnings("ignore")
    nlp = spacy.load('en_core_web_md')
    tokens = nlp(row['aspects_x'] + " | " + row['aspects_y'])
    sep_index = [token.i for token in tokens if token.text == '|'][0]
    token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
    row['aspects_x_len'] = len(token1)
    row['aspects_y_len'] = len(token2)
    warnings.filterwarnings("error")
    try:
#         print(token2.similarity(token1))
        row['score'] = token2.similarity(token1)
    except:
        row['score'] = 0
    row['predicted_by_cosine'] = True
    
    return row

In [ ]:
import nltk
def label_implicit_matching_pair(row):
    token1 = row['aspects_x'][0]
    token2 = row['aspects_y'][0]
    
    # Best
    if row['aspects_x_len'] >= row['aspects_y_len']:
        if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
            row['label'] = 1
    return row

In [ ]:
def do_eval(df):
    comment_df, kp_df, labels_df = prepare_comment_kp_label_input(df)
    predictions_df = comment_kp_absa_match(comment_df, kp_df)

    merged_df = pd.merge(predictions_df, labels_df, how="left", on=["comment_id", "key_point_id"])
    merged_df.loc[merged_df['key_point_id'] == "dummy_id", 'label'] = 0
    merged_df["label_strict"] = merged_df["label"].fillna(0)
    merged_df["label_relaxed"] = merged_df["label"].fillna(1)
    
    precisions = [(t, get_ap(group, "label_strict")) for t, group in merged_df.groupby(["topic"])]
    
    return merged_df, precisions

# Evaluation

## All Comments in test data

In [ ]:
merged_df, precisions = do_eval(df)

# Quantitative KP Coverage and Precision/Recall

In [102]:
merged_df[merged_df['topic'] == 'Arts & Entertainment'].merge(kp_df, on=['key_point_id']).columns

Index(['comment_id', 'topic_x', 'full_comment', 'num_of_aspects', 'aspects_x',
       'opinions_x', 'opinion_aspect_pairs_x', 'sentiments_x', 'attributes_x',
       'key_point_id', 'key_point_x', 'aspects_y', 'opinions_y',
       'opinion_aspect_pairs_y', 'sentiments_y', 'attributes_y', 'score',
       'predicted_by_cosine', 'label', 'label_strict', 'label_relaxed',
       'topic_y', 'key_point_y', 'aspects', 'opinions', 'opinion_aspect_pairs',
       'sentiments', 'attributes'],
      dtype='object')

In [150]:
from sklearn.metrics import recall_score, precision_score

def get_top_kp_coverages(merged_df, df, category, top=None, selected_kps=[]):
    comment_df, kp_df, labels_df = prepare_comment_kp_label_input(df)
    kp_df = kp_df.drop(columns=['key_point'])

    valid_df = df[(df['label'] == 1) & (df['topic'] == category)]
    valid_kp = valid_df['full_key_point'].value_counts()
    display(valid_kp)
    
    top_kp = merged_df[merged_df['topic'] == category].merge(kp_df, on=['key_point_id'])['key_point'].value_counts()
    result = top_kp[top_kp.index.isin(valid_kp.index.tolist())]
    if top != None:
        result = result[result.index.isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        result = result[result.index.isin(selected_kps)]
    
    if top != None:
        valid_df = df[df['full_key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        valid_df = valid_df[valid_df['full_key_point'].isin(selected_kps)]
    
    pred_df = merged_df[merged_df['topic'] == category].merge(kp_df, on=['key_point_id'])
    if top != None:
        pred_df = pred_df[pred_df['key_point'].isin(valid_kp.head(top).index)]
    elif len(selected_kps) > 0:
        pred_df = pred_df[pred_df['key_point'].isin(selected_kps)]
        
    return valid_df, pred_df

In [151]:
def calculate_precision_recall(valid_df, pred_df):
    pred_df['pred_label'] = 1
    
    # Calculate Precision
    new_valid_df = valid_df.merge(pred_df[['comment_id', 'key_point_id', 'pred_label']], how='right')
    new_valid_df['label'] = new_valid_df['label'].fillna(0)
    precision = precision_score(new_valid_df['label'], new_valid_df['pred_label'])

    # Calculate Recall
    new_valid_df = valid_df.merge(pred_df[['comment_id', 'key_point_id', 'pred_label']], how='left')
    new_valid_df['pred_label'] = new_valid_df['pred_label'].fillna(0)
    recall = recall_score(new_valid_df['label'], new_valid_df['pred_label'])
    
    return precision, recall

In [157]:
def generate_coverage_table(valid_df, pred_df): 
    coverage_compare_table = pred_df['key_point'].value_counts().reset_index()
    coverage_compare_table.columns = ['Key Points', 'Predicted Coverage']
    coverage_compare_table['Actual Coverage'] = valid_df['full_key_point'].value_counts().tolist()
    return coverage_compare_table

## Arts & Entertainment

In [158]:
selected_kps = ['Friendly and helpful staff .', 'Seats are adequately comfortable .   ', 'Horrible customer service .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Arts & Entertainment', None, selected_kps)

Friendly and helpful staff .                14
Awesome service and employees .             10
It had wonderful acoustics .                 9
Superior customer service !                  8
Seats are adequately comfortable .           4
Horrible customer service .                  3
Very talented musicians !                    2
Worth the money .                            2
lines were short and quick .                 2
The acoustics are poor .                     2
Terrible administration and management .     2
Rude and Unprofessional staff .              2
Great for conventions and concerts ! !       2
Incredibly talented performers - wow !       2
Parking was fast and FREE !                  1
Front seats worth every penny !              1
Awesome lighting , and decor .               1
Nice employees ... not to pushy .            1
This theatre was filthy .                    1
Name: full_key_point, dtype: int64

In [159]:
generate_coverage_table(valid_df, pred_df)

,Key Points,Predicted Coverage,Actual Coverage
0,Friendly and helpful staff .,10,14
1,Seats are adequately comfortable .,4,4
2,Horrible customer service .,3,3


In [160]:
calculate_precision_recall(valid_df, pred_df)

(0.9411764705882353, 0.7619047619047619)

## Automotive

In [161]:
selected_kps = ['They have excellent customer service .', 'The employees here are wonderful !', 'Very professional staff .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Automotive', None, selected_kps)

They have excellent customer service .    29
Very professional staff .                 13
The employees here are wonderful !        13
Very courteous staff .                    10
FAST and FRIENDLY SERVICE ! ! !            6
TERRIBLE CUSTOMER SERVICE !                5
Their work is Impeccable .                 4
The cashiers are always friendly .         3
Store was clean and organized .            3
Extremely poor service .                   2
Prices were fair and reasonable .          2
Over priced and overrated .                1
Overall just poor quality .                1
Unbeatable Professional Service !          1
Name: full_key_point, dtype: int64

In [162]:
generate_coverage_table(valid_df, pred_df)

ValueError: Length of values (3) does not match length of index (2)

In [163]:
calculate_precision_recall(valid_df, pred_df)

(0.8333333333333334, 0.18181818181818182)

## Beauty & Spas

In [164]:
selected_kps = ['Staff is friendly and accommodating .', 'Customer service- EXCELLENT !', 'Amazing & professional service .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, "Beauty & Spas", None, selected_kps)

Staff is friendly and accommodating .    18
Amazing & professional service .         14
Customer service- EXCELLENT !            13
Very professionally run business .        9
Artists are highly skilled .              7
Super clean and friendly salon .          6
Very neat and painless service .          3
POOR CUSTOMER SERVICE .                   2
Super comfy and great massage .           2
Prices were fair and reasonable .         2
This place is TERRIFIC .                  1
Name: full_key_point, dtype: int64

In [165]:
generate_coverage_table(valid_df, pred_df)

,Key Points,Predicted Coverage,Actual Coverage
0,Staff is friendly and accommodating .,13,18
1,Customer service- EXCELLENT !,7,14
2,Amazing & professional service .,3,13


In [166]:
calculate_precision_recall(valid_df, pred_df)

(0.8695652173913043, 0.4444444444444444)

## Hotels

In [167]:
selected_kps = ['Friendly and helpful staff .', 'Clean and comfortable rooms .', 'The ambience is wonderfully peaceful .']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Hotels', None, selected_kps)

Friendly and helpful staff .              21
Clean and comfortable rooms .             13
Excellent customer service .               7
The service is impeccable .                5
Very peaceful and relaxing location .      4
Great desk service , too .                 4
Extremely poor service .                   3
The beds are extremely comfortable .       2
Pool area clean and refreshing .           2
Price way overvalued .                     1
The ambience is wonderfully peaceful .     1
Horribly rude staff .                      1
Front desk staff was inept .               1
Room decor is outdated .                   1
Wifi was very reliable .                   1
Valet is convenient and fast .             1
Parking is easy .                          1
Restaurant service absolutely sucks .      1
Beds were extremely uncomfortable .        1
Great cocktail list too .                  1
Name: full_key_point, dtype: int64

In [168]:
generate_coverage_table(valid_df, pred_df)

,Key Points,Predicted Coverage,Actual Coverage
0,Friendly and helpful staff .,16,21
1,Clean and comfortable rooms .,12,13
2,The ambience is wonderfully peaceful .,2,1


In [169]:
calculate_precision_recall(valid_df, pred_df)

(0.9333333333333333, 0.8)

## Restaurants

In [170]:
selected_kps = ['Staff was courteous and accommodating .', 
                'Fresh food , using local produce .', 'The service here was exceptional .  ']
valid_df, pred_df = get_top_kp_coverages(merged_df, df, 'Restaurants', None, selected_kps)

Staff was courteous and accommodating .       19
Customer Service is Excellent .               14
Employees are friendly and attentive .        14
Service was prompt and friendly .             11
Delicious and expertly prepared food .         7
Ambiance is casual and comfortable .           6
Fresh food , using local produce .             5
The service here was exceptional .             5
Really delicious food , well balanced !        4
Fantastic drink selection .                    3
Generous sized portions .                      3
The food is consistently EXCELLENT !           3
AMAZING AUTHENTIC FLAVOR !                     3
Plenty of coffees and teas .                   2
Way over priced .                              2
Great variety of menu items .                  2
Prices are fair and reasonable .               2
Great beer and wine selection .                1
Lots of outdoor seating and                    1
The quality has diminished significantly .     1
Name: full_key_point

In [171]:
generate_coverage_table(valid_df, pred_df)

,Key Points,Predicted Coverage,Actual Coverage
0,Staff was courteous and accommodating .,11,19
1,"Fresh food , using local produce .",8,5
2,The service here was exceptional .,5,5


In [172]:
calculate_precision_recall(valid_df, pred_df)

(0.75, 0.6206896551724138)